In [19]:
import numpy as np
from functools import partial
import matplotlib.pyplot as plt
import seaborn as sns
import time
import numba
from numba import njit

In [27]:
def sghmc_with_grad(U_grad, theta_init, M, C, V_hat, epsilon, T, m):
    """
    Stochastic Gradient Hamiltonian Monte Carlo Sampling.
    Based on Chen, Tianqi, Emily Fox, and Carlos Guestrin (2014)
    --------------------
    
    Dimensions
    -----------
    d: number of parameters
    T: length of samples
    
    Input
    ------
    U_grad: callable 
        Stochastic gradient estimates of posterior density with respect to distribution parameters 
        'U_grad_tilde(D, logp_data_grad, logp_prior_grad, mb_size, theta)' when the gradient is unknown
    
    theta_init: d-by-1 np array
        The inital sampling point
        
    M: d-by-d np array
        A mass matrix
    
    C: d-by-d np array
        A user specified friction term, should be greater than B_hat = 0.5*epsilon*V_hat 
        in the sense of in the sense of positive semi-definiteness
    
    V_hat: d-by-d np array
        Empirical Fisher information of theta
        
    epsilon: float
        Step size
    
    T: int
        Number of samples drawn from the desired distribution
        
    m: int
        Number of steps for each draw
    
    
    Output
    ------
    theta_s: T-by-d np array
        Draws sampled from the desired distrition
    
    r_s: T-by-d np array
        Draws of the momentun variables
    
    """
    d = len(theta_init)
    theta_s = np.zeros((T, d))
    r_s = np.zeros((T, d))
    theta_s[0] = theta_init
    M_inv = np.linalg.inv(M)
    B_hat = 0.5*epsilon*V_hat
    
    if d > 1:
        sd = np.linalg.cholesky(2*epsilon*(C-B_hat))
        r_s = np.random.multivariate_normal(np.zeros(d),M, size = T)
    elif d==1:
        sd = np.sqrt(2*epsilon*(C-B_hat))
        r_s = np.sqrt(M)*np.random.randn(T).reshape(T,1)
    
    for t in range(T-1):
        theta0 = theta_s[t]
        r0 = r_s[t]
        for i in range(m):
            theta0 = theta0 + epsilon*np.dot(M_inv,r0)
            r0 = r0 - epsilon*U_grad(theta0) - epsilon*np.dot(np.dot(C,M_inv),r0) +  np.dot(sd,np.random.randn(d))
        theta_s[t+1] = theta0
    
    return [theta_s,r_s]

In [28]:
## potential energy function
U = lambda x:  -2*x**2 + x**4
gradU =  lambda x: -4 * x +  4 * x**3 


## parameters
theta_init = np.array([0])
M = np.eye(1)
epsilon=0.1
T=10000
m=50
V = np.array([4])
C = np.array([3]) 
V_hat = np.array([4])


## true distribution
xs = np.linspace(-2,2,200)
ys = np.exp(-U(xs))
ys = (ys/sum(ys))/(xs[1]-xs[0])

In [ ]:
%%time
theta, r = sghmc_with_grad(gradU, theta_init, M, C, V_hat, epsilon, T, m)

In [29]:
%prun -q -D sghmc1.prof sghmc_with_grad(gradU, theta_init, M, C, V_hat, epsilon, T, m)

 
*** Profile stats marshalled to file 'sghmc1.prof'. 


In [30]:
import pstats
p = pstats.Stats('sghmc1.prof')
p.sort_stats('tottime').print_stats()
pass

Sun Apr 25 19:29:48 2021    sghmc1.prof

         6999330 function calls in 25.250 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    8.576    8.576   25.250   25.250 <ipython-input-27-62c79aa19318>:1(sghmc_with_grad)
  1999800    6.007    0.000   11.605    0.000 <__array_function__ internals>:2(dot)
   499950    3.793    0.000    3.793    0.000 <ipython-input-28-aa50056b54bc>:3(<lambda>)
  1999801    3.767    0.000    3.767    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
  1999800    1.832    0.000    1.832    0.000 /opt/conda/lib/python3.6/site-packages/numpy/core/multiarray.py:707(dot)
   499951    1.275    0.000    1.275    0.000 {method 'randn' of 'numpy.random.mtrand.RandomState' objects}
        2    0.001    0.000    0.001    0.000 {built-in method numpy.zeros}
        1    0.000    0.000    0.000    0.000 /opt/conda/lib/python3.6/site-packages/numpy/linalg/linalg.py:482(in

In [31]:
def sghmc_with_grad(U_grad, theta_init, M, C, V_hat, epsilon, T, m):
    """
    Stochastic Gradient Hamiltonian Monte Carlo Sampling.
    Based on Chen, Tianqi, Emily Fox, and Carlos Guestrin (2014)
    """
    d = len(theta_init)
    theta_s = np.zeros((T, d))
    r_s = np.zeros((T, d))
    theta_s[0] = theta_init
    M_inv = np.linalg.inv(M)
    B_hat = 0.5*epsilon*V_hat
    
    if d > 1:
        sd = np.linalg.cholesky(2*epsilon*(C-B_hat))
        r_s = np.random.multivariate_normal(np.zeros(d),M, size = T)
    elif d==1:
        sd = np.sqrt(2*epsilon*(C-B_hat))
        r_s = np.sqrt(M)*np.random.randn(T).reshape(T,1)
    
    for t in range(T-1):
        theta0 = theta_s[t]
        r0 = r_s[t]
        for i in range(m):
            theta0 = theta0 + epsilon*M_inv@r0
            r0 = r0 - epsilon*U_grad(theta0) - epsilon*C@M_inv@r0 + sd@np.random.randn(d)
        theta_s[t+1] = theta0
    
    return [theta_s,r_s]

In [33]:
%prun -q -D sghmc2.prof sghmc_with_grad(gradU, theta_init, M, C, V_hat, epsilon, T, m)

 
*** Profile stats marshalled to file 'sghmc2.prof'. 


In [34]:
import pstats
p = pstats.Stats('sghmc2.prof')
p.sort_stats('tottime').print_stats()
pass

Sun Apr 25 19:31:53 2021    sghmc2.prof

         999930 function calls in 13.951 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    8.921    8.921   13.951   13.951 <ipython-input-31-247da866bc42>:1(sghmc_with_grad)
   499950    3.742    0.000    3.742    0.000 <ipython-input-28-aa50056b54bc>:3(<lambda>)
   499951    1.289    0.000    1.289    0.000 {method 'randn' of 'numpy.random.mtrand.RandomState' objects}
        2    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        1    0.000    0.000    0.000    0.000 /opt/conda/lib/python3.6/site-packages/numpy/linalg/linalg.py:482(inv)
        1    0.000    0.000   13.951   13.951 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 /opt/conda/lib/python3.6/site-packages/numpy/linalg/linalg.py:144(_commonType)
        1    0.000    0.000   13.951   13.951 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 /op

In [35]:
def sghmc_with_grad_numba1(U_grad, theta_init, M, C, V_hat, epsilon, T, m):
    """
    Stochastic Gradient Hamiltonian Monte Carlo Sampling.
    Based on Chen, Tianqi, Emily Fox, and Carlos Guestrin (2014)
    """
    d = len(theta_init)
    theta_s = np.zeros((T, d))
    r_s = np.zeros((T, d))
    theta_s[0] = theta_init
    M_inv = np.linalg.inv(M)
    B_hat = 0.5*epsilon*V_hat
    
    if d > 1:
        sd = np.linalg.cholesky(2*epsilon*(C-B_hat))
        r_s = np.random.multivariate_normal(np.zeros(d),M, size = T)
    elif d==1:
        sd = np.sqrt(2*epsilon*(C-B_hat))
        r_s = np.sqrt(M)*np.random.randn(T).reshape(T,1)
    
    U_grad_jit = numba.njit(U_grad)
    
    for t in range(T-1):
        theta0 = theta_s[t]
        r0 = r_s[t]
        for i in range(m):
            theta0 = theta0 + epsilon*M_inv@r0
            r0 = r0 - epsilon*U_grad_jit(theta0) - epsilon*C@M_inv@r0 + sd@np.random.randn(d)
        theta_s[t+1] = theta0
    
    return [theta_s,r_s]

In [37]:
%%timeit
theta, r = sghmc_with_grad_numba1(gradU, theta_init, M, C, V_hat, epsilon, T, m)

8.68 s ± 44.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
def sghmc_with_grad_numba2(U_grad, theta_init, M, C, V_hat, epsilon, T, m):
    """
    Stochastic Gradient Hamiltonian Monte Carlo Sampling.
    Based on Chen, Tianqi, Emily Fox, and Carlos Guestrin (2014)
    """
    d = len(theta_init)
    theta_s = np.zeros((T, d))
    r_s = np.zeros((T, d))
    theta_s[0] = theta_init
    M_inv = np.linalg.inv(M)
    B_hat = 0.5*epsilon*V_hat
    
    if d > 1:
        sd = np.linalg.cholesky(2*epsilon*(C-B_hat))
        r_s = np.random.multivariate_normal(np.zeros(d),M, size = T)
    elif d==1:
        sd = np.sqrt(2*epsilon*(C-B_hat))
        r_s = np.sqrt(M)*np.random.randn(T).reshape(T,1)
    
    U_grad_jit = numba.njit(U_grad)
    
    @numba.njit
    def update(x,y):
        x = x + epsilon*M_inv@y
        y = y - epsilon*U_grad_jit(x) - epsilon*C@M_inv@y + sd@np.random.randn(d)
        return [x,y]
    
    for t in range(T-1):
        theta0 = theta_s[t]
        r0 = r_s[t]
        for i in range(m):
            theta0, r0 = update(theta0, r0)
        theta_s[t+1] = theta0
    
    return [theta_s,r_s]

In [41]:
%%timeit
theta, r = sghmc_with_grad_numba2(gradU, theta_init, M, C, V_hat, epsilon, T, m)

1.96 s ± 18.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
def U_grad_tilde(theta, data, logp_data_grad, logp_prior_grad, mb_size):
    """
    Stochastic gradient estimates of posterior density with respect to distribution parameters
    Based on a minibatch D_hat sampled uniformly at random from D
    ------------------------
    
    Dimensions
    -----------
    n: number of observations from the data
    m: dimension of the data

    Input
    -----
    D: n-by-m np array
        Dataset
        
    logp_data_grad: callable 'logp_data_grad(data, theta)'
        Gradient of likelihood of the data with respect to distribution parameters
    
    logp_prior_grad: callable 'logp_prior_grad(theta)'
        Gradient of prior with respect to distribution parameters
    
    mb_size: int
        Size of the minibatch
    
    theta: d-by-1 np array
        Distribution parameters
    
    Output
    -----
    U_tilde: d-by-1 np array
        Stochastic gradient estimates of posterior density with respect to distribution parameters
    """
    n = data.shape[0]
    data_hat = data[np.random.choice(range(n), size = mb_size, replace = False)]
    U_tilde = -(n/mb_size)*logp_data_grad(data_hat, theta) - logp_prior_grad(theta)
    return U_tilde

In [67]:
def sghmc_with_data(data, logp_data_grad, logp_prior_grad, mb_size, theta_init, M, C, V_hat, epsilon, T, m):
    """
    Stochastic Gradient Hamiltonian Monte Carlo Sampling.
    Based on Chen, Tianqi, Emily Fox, and Carlos Guestrin (2014)
    """
    d = theta_init.shape[0]
    theta_s = np.zeros((T, d))
    r_s = np.zeros((T, d))
    theta_s[0] = theta_init
    M_inv = np.linalg.inv(M)
    B_hat = 0.5*epsilon*V_hat
    
    
    n = data.shape[0]
    data_hat = data[np.random.choice(range(n), size = mb_size, replace = False)]
    
    if d > 1:
        sd = np.linalg.cholesky(2*epsilon*(C-B_hat))
        r_s = np.random.multivariate_normal(np.zeros(d),M, size = T)
    elif d==1:
        sd = np.sqrt(2*epsilon*(C-B_hat))
        r_s = np.sqrt(M)*np.random.randn(T).reshape(T,1)

    
    for t in range(T-1):
        theta0 = theta_s[t]
        r0 = r_s[t]
        for i in range(m):
            theta0 = theta0 + epsilon*M_inv@r0
            r0 = r0 + epsilon*((n/mb_size)*logp_data_grad(data_hat, theta0) +logp_prior_grad(theta0)) - epsilon*C@M_inv@r0 + sd@np.random.randn(d)
    
    return [theta_s,r_s]

In [68]:
p = 5
true_theta = np.arange(p)
size = 10000
X = np.random.randn(size,p)
y = np.dot(X,true_theta) + np.random.randn(size)
data = np.c_[y,X]

theta_init = np.zeros(p)
M = np.eye(p)
C = 13*np.eye(p)
V_hat = 0
T = 1000
m = 50
epsilon = 0.0001
mb_size=1000

In [69]:
logp_prior_grad = lambda theta: -theta # prior N(0,1)

def logp_data_grad(data, theta):
    """
    log likelihood of a linear regression 
    --------------
    
    Assume that the first column of the data is the predicted variable
    """
    X = data[:,1:]
    y = data[:,0]
    return X.T@y - X.T@X@theta

In [54]:
gradU = partial(U_grad_tilde, data=data, logp_data_grad=logp_data_grad, logp_prior_grad=logp_prior_grad, mb_size=1000)

In [55]:
%%time
theta5, r5 = sghmc_with_grad(gradU, theta_init, M, C, V_hat, epsilon, T, m)

CPU times: user 1min 21s, sys: 228 ms, total: 1min 22s
Wall time: 1min 21s


In [72]:
%%timeit
theta, r = sghmc_with_data(data, logp_data_grad, logp_prior_grad, mb_size, theta_init, M, C, V_hat, epsilon, T, m)

2.12 s ± 46.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [70]:
%prun -q -D sghmc3.prof sghmc_with_data(data, logp_data_grad, logp_prior_grad, mb_size, theta_init, M, C, V_hat, epsilon, T, m)

 
*** Profile stats marshalled to file 'sghmc3.prof'. 


In [71]:
import pstats
p = pstats.Stats('sghmc3.prof')
p.sort_stats('tottime').print_stats()
pass

Sun Apr 25 20:34:52 2021    sghmc3.prof

         149996 function calls (149991 primitive calls) in 2.541 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    49950    1.191    0.000    1.191    0.000 <ipython-input-69-b807f05dee81>:3(logp_data_grad)
        1    1.027    1.027    2.541    2.541 <ipython-input-67-44e6294cb69c>:1(sghmc_with_data)
    49950    0.240    0.000    0.240    0.000 {method 'randn' of 'numpy.random.mtrand.RandomState' objects}
    49950    0.080    0.000    0.080    0.000 <ipython-input-69-b807f05dee81>:1(<lambda>)
        1    0.002    0.002    0.002    0.002 {method 'choice' of 'numpy.random.mtrand.RandomState' objects}
        1    0.001    0.001    0.001    0.001 {method 'multivariate_normal' of 'numpy.random.mtrand.RandomState' objects}
        1    0.000    0.000    0.000    0.000 /opt/conda/lib/python3.6/site-packages/numpy/linalg/linalg.py:482(inv)
     11/6    0.000    0.000    0.001    0.00

In [73]:
def sghmc_with_data_numba(data, logp_data_grad, logp_prior_grad, mb_size, theta_init, M, C, V_hat, epsilon, T, m):
    """
    Stochastic Gradient Hamiltonian Monte Carlo Sampling.
    Based on Chen, Tianqi, Emily Fox, and Carlos Guestrin (2014)
    """
    d = theta_init.shape[0]
    theta_s = np.zeros((T, d))
    r_s = np.zeros((T, d))
    theta_s[0] = theta_init
    M_inv = np.linalg.inv(M)
    B_hat = 0.5*epsilon*V_hat
    
    
    logp_data_grad_jit = numba.njit(logp_data_grad)
    logp_prior_grad_jit = numba.njit(logp_prior_grad)
    
    n = data.shape[0]
    data_hat = data[np.random.choice(range(n), size = mb_size, replace = False)]
    
    if d > 1:
        sd = np.linalg.cholesky(2*epsilon*(C-B_hat))
        r_s = np.random.multivariate_normal(np.zeros(d),M, size = T)
    elif d==1:
        sd = np.sqrt(2*epsilon*(C-B_hat))
        r_s = np.sqrt(M)*np.random.randn(T).reshape(T,1)
    
    @numba.njit
    def update(x,y):
        x = x + epsilon*M_inv@y
        y = y + epsilon*((n/mb_size)*logp_data_grad_jit(data_hat, x) +logp_prior_grad_jit(x)) - epsilon*C@M_inv@y + sd@np.random.randn(d)
        return [x,y]
    
    for t in range(T-1):
        theta0 = theta_s[t]
        r0 = r_s[t]
        for i in range(m):
            theta0, r0 = update(theta0, r0)
        theta_s[t+1] = theta0
    
    return [theta_s,r_s]

In [74]:
%%time
theta, r = sghmc_with_data_numba(data, logp_data_grad, logp_prior_grad, mb_size, theta_init, M, C, V_hat, epsilon, T, m)

<ipython-input-73-7f9c7a93f67c>:30: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (readonly array(float64, 2d, A), readonly array(float64, 1d, A))
  y = y + epsilon*((n/mb_size)*logp_data_grad_jit(data_hat, x) +logp_prior_grad_jit(x)) - epsilon*C@M_inv@y + sd@np.random.randn(d)
<ipython-input-73-7f9c7a93f67c>:30: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (readonly array(float64, 2d, A), readonly array(float64, 2d, A))
  y = y + epsilon*((n/mb_size)*logp_data_grad_jit(data_hat, x) +logp_prior_grad_jit(x)) - epsilon*C@M_inv@y + sd@np.random.randn(d)


CPU times: user 3.35 s, sys: 284 ms, total: 3.64 s
Wall time: 3.17 s


In [75]:
p = 5
true_theta = np.arange(p)
size = 10000
X = np.random.randn(size,p)
y = np.dot(X,true_theta) + np.random.randn(size)
data = np.c_[y,X]

theta_init = np.zeros(p)
M = np.eye(p)
C = 13*np.eye(p)
V_hat = 0
T = 10000
m = 50
epsilon = 0.0001
mb_size=1000

In [76]:
%%time
theta, r = sghmc_with_data(data, logp_data_grad, logp_prior_grad, mb_size, theta_init, M, C, V_hat, epsilon, T, m)

CPU times: user 21.6 s, sys: 664 ms, total: 22.2 s
Wall time: 21.3 s


In [77]:
%%time
theta, r = sghmc_with_data_numba(data, logp_data_grad, logp_prior_grad, mb_size, theta_init, M, C, V_hat, epsilon, T, m)

<ipython-input-73-7f9c7a93f67c>:30: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (readonly array(float64, 2d, A), readonly array(float64, 1d, A))
  y = y + epsilon*((n/mb_size)*logp_data_grad_jit(data_hat, x) +logp_prior_grad_jit(x)) - epsilon*C@M_inv@y + sd@np.random.randn(d)
<ipython-input-73-7f9c7a93f67c>:30: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (readonly array(float64, 2d, A), readonly array(float64, 2d, A))
  y = y + epsilon*((n/mb_size)*logp_data_grad_jit(data_hat, x) +logp_prior_grad_jit(x)) - epsilon*C@M_inv@y + sd@np.random.randn(d)


CPU times: user 19.4 s, sys: 416 ms, total: 19.8 s
Wall time: 19.1 s
